# 품질검사성적서 등록 목록

In [1]:
from common import commonFunc as cf
import pandas as pd
pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "품질검사성적서 등록 목록"
APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/품질검사성적서 등록 목록/selectIoPmQtscList.csv'

In [2]:
targetData = metadata.loc[metadata.자료명==DATANAME]

##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]

In [3]:
JSONKEY = "items"
DUMMY = 0

In [4]:
#### 페이지번호 설정 ###
PAGEYN=1
if REQPARAM.count("pageyn") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1

In [5]:
#임시 
imsiDf = pd.read_csv(\
         "../output/건설사업정보시스템/건설자재 품질검사 등록정보/selectIoPmQtlTsitStsList.csv",low_memory=False, encoding="ms949")

In [6]:
dfCombi = imsiDf.loc[:,["cstrnm"]].drop_duplicates()
dfCombi.reset_index(inplace=True, drop=True)
dfCombiLen = dfCombi.shape[0]
dfCombiLen

74882

In [7]:
resultDfMerged = pd.DataFrame()

In [8]:
import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
endYr = date.strftime("%Y")
startYr = 2001
endYr

'2022'

In [9]:
# 기준년도 산정
# 2001년 부터 데이터 존재
# startYr = "2001"
# dateRange = list(map(int,pd.period_range(startYr,endYr,freq='Y').strftime('%Y').tolist()))
# # dateRange

## 품질검사성적서 등록정보 전용 스크랩 함수

In [10]:
import requests
from lxml import html
from bs4 import BeautifulSoup 
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import os

STDENCODING="utf-8"
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
### 파마리터정의: 
###   - inurl: 메타정보의 "URL"컬럼값 (예: https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do )
###   - inSiteName: 메타정보의 "자료대상" (예: 건설사업정보시스템)
###   - inDataName: 메타정보의 "자료명" (예: 공사정보 목록)
###   - inServiceName: 메타정보의 "기본키" (예: serviceKey + Format)
###   - inParam: 메타정보의 "파라미터 정보" (예: 페이지 파라미터 존재 시 1 값")
###   - inPageYn: 메타정보의 "페이지 정보" (예: 페이지 파라미터 존재 시 1 값")
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
def myscrapy(inUrl, inSiteName, inDataName, inServiceName, inParam, inPageYn, jsonkey="items", dummy=0, inType="jsonabnormal",x=-1):
    try:
        emptyPd = pd.DataFrame()
        i=1
        while True:
            print("{} page scraping start".format(i))
            
            if(inPageYn==1):
                inParam["pageNo"] = i
            queryParams = '?' + urlencode(inParam)
            requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
            response = requests.get(inUrl+queryParams,verify=False)
            response.encoding=STDENCODING
            rowData = pd.DataFrame()
            if(inType=="jsonabnormal"):
                # 비정상 데이터는 response 섹션이 없음
                if(response.json().get('response') == None):
                    jsondata = response.json()["header"]["resultMsg"]
                    if( jsondata != "NORMAL_SERVICE"):
                        print("SERVER ERROR ",jsondata)
                        break
                    
#               여기서부터는 데이터는 있는데, totalcount>0 경우는 items 정보에 세부정보가 더 있음
               
                jsondata = response.json()["response"]["body"][jsonkey]
                if( jsondata == []):
                    print("{} page is empty".format(i))
                    break
                if( jsonkey == "detail1"):
                    jsondata["index"]=[0]
                rowData = pd.DataFrame(jsondata)
                emptyPd = emptyPd.append(rowData)

                print(">totalCount "+str(response.json()["response"]["body"]["totalCount"]))
                if(0 < int(response.json()["response"]["body"]["totalCount"])):
                    global g_totalCount
                    if g_totalCount[x] == 0:
                        g_totalCount[x] = int(response.json()["response"]["body"]["totalCount"])
                    
            else:
                print("Error")          
     
            if(inPageYn == 0):
                print("{} no pageNo".format(inPageYn))
                break
            i = i+1

        emptyPd.columns = emptyPd.columns.str.lower()
        emptyPd.shape
#         print("dataframe{}, param:{} rows: {} completed".format(inDataName,inParam, emptyPd.shape[1] )     )
        return emptyPd       
    except Exception as e:
            print(e)     

In [11]:
numOfThread = 50

resultDfMerged = list(pd.DataFrame() for x in range(numOfThread))

g_totalCount = list(range(numOfThread))

In [15]:
def threadReq(startIdx, endIdx,x):
    numOfRows = 1000 # 서버에서 최대 응답 가능한 row임
    resultDf = pd.DataFrame()
    for i in range(startIdx,endIdx):
        searchCstrNm = dfCombi.loc[i].cstrnm
        pageNo = 1
        print(searchCstrNm)
        global g_totalCount
        g_totalCount[x] = 0
        while True:
            BASEPARAM={"serviceKey":APIKEY,"searchCstrNm": searchCstrNm, "searchRpcdIsBgDt": str(startYr)+"0101","searchRpcdIsEdDt": str(endYr)+"1231", "sortField": "tstBgDt","pageNo": pageNo, 'numOfRows':numOfRows, "type":"json"}
            print("BASEPARAM ",BASEPARAM)
            resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY, "jsonabnormal",x)
            if resultDf.empty: # 정상 데이터가 없는 경우
                print("No Data")
                break
            else:
                resultDfMerged[x] = resultDfMerged[x].append(resultDf) 
                g_totalCount[x] -= numOfRows
                print("x : ",x," g_totalCount : ", g_totalCount[x], " numOfRows : ", numOfRows)
                if(g_totalCount[x] <= 0):
                    break
                pageNo += 1

In [13]:
import threading

resultDfLastMerged = pd.DataFrame()
section = list(range(0, dfCombiLen, int(dfCombiLen/numOfThread))) # we have "numåOfThread +1" sections
section

[0,
 748,
 1496,
 2244,
 2992,
 3740,
 4488,
 5236,
 5984,
 6732,
 7480,
 8228,
 8976,
 9724,
 10472,
 11220,
 11968,
 12716,
 13464,
 14212,
 14960,
 15708,
 16456,
 17204,
 17952,
 18700,
 19448,
 20196,
 20944,
 21692,
 22440,
 23188,
 23936,
 24684,
 25432,
 26180,
 26928,
 27676,
 28424,
 29172,
 29920,
 30668,
 31416,
 32164,
 32912,
 33660,
 34408,
 35156,
 35904,
 36652,
 37400,
 38148,
 38896,
 39644,
 40392,
 41140,
 41888,
 42636,
 43384,
 44132,
 44880,
 45628,
 46376,
 47124,
 47872,
 48620,
 49368,
 50116,
 50864,
 51612,
 52360,
 53108,
 53856,
 54604,
 55352,
 56100,
 56848,
 57596,
 58344,
 59092,
 59840,
 60588,
 61336,
 62084,
 62832,
 63580,
 64328,
 65076,
 65824,
 66572,
 67320,
 68068,
 68816,
 69564,
 70312,
 71060,
 71808,
 72556,
 73304,
 74052,
 74800]

In [14]:
# dfCombiLen = 6 # for test
if dfCombiLen < numOfThread:
    print("데이터의 갯수가 적어서 쓰레드를 시작할 수 없습니다.")
else:
    x_ls =list(range(numOfThread))
    thread_list = []
    results = []
    idx = 0
    for x in x_ls:
        startIdx = section[x]
        endIdx =  section[x+1] -1 if (x+1 <= numOfThread -1) else dfCombiLen - 1
        print("start ",startIdx, " end ",endIdx)
        mythread = threading.Thread(target=threadReq, args=(startIdx, endIdx+1,idx))
        idx += 1
        thread_list.append(mythread)
    for i in thread_list:
        i.start()
    for i in thread_list:
        i.join()
    for i in range(0,numOfThread):
        resultDfLastMerged = resultDfLastMerged.append(resultDfMerged[i]) 



start  0  end  747
start  748  end  1495
start  1496  end  2243
start  2244  end  2991
start  2992  end  3739
start  3740  end  4487
start  4488  end  5235
start  5236  end  5983
start  5984  end  6731
start  6732  end  7479
start  7480  end  8227
start  8228  end  8975
start  8976  end  9723
start  9724  end  10471
start  10472  end  11219
start  11220  end  11967
start  11968  end  12715
start  12716  end  13463
start  13464  end  14211
start  14212  end  14959
start  14960  end  15707
start  15708  end  16455
start  16456  end  17203
start  17204  end  17951
start  17952  end  18699
start  18700  end  19447
start  19448  end  20195
start  20196  end  20943
start  20944  end  21691
start  21692  end  22439
start  22440  end  23187
start  23188  end  23935
start  23936  end  24683
start  24684  end  25431
start  25432  end  26179
start  26180  end  26927
start  26928  end  27675
start  27676  end  28423
start  28424  end  29171
start  29172  end  29919
start  29920  end  30667
start  

경기관광공사   2001
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '경기관광공사', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
대우건설㈜ ,㈜영동건설,㈜신태진건설대양공영   2001
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '대우건설㈜ ,㈜영동건설,㈜신태진건설', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
 세양엔지니어링(주  2001
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '대양공영', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start   2001
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '세양엔지니어링(주', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField'

홍우전업   2001
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '홍우전업', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '지케이기술연구소', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
(주)엠포종합건설   2001
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)엠포종합건설', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
영성기공   2001
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '영성기공', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 10

1 page is empty1 page is empty
No Data
(유)동무건설   2002
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(유)동무건설', 'searchRpcdIsBgDt': '20020101', 'searchRpcdIsEdDt': '20021231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start

No Data
승효건설(주)   2002
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '승효건설(주)', 'searchRpcdIsBgDt': '20020101', 'searchRpcdIsEdDt': '20021231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty1 page is empty
No Data
웅진개발   2002
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '웅진개발', 'searchRpcdIsBgDt': '20020101', 'searchRpcdIsEdDt': '20021231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start

No Data
(주)청암종합건설   2002
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)청암종합건설', 

1 page is empty
No Data
뉴삼원종합건설㈜   2002
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '뉴삼원종합건설㈜', 'searchRpcdIsBgDt': '20020101', 'searchRpcdIsEdDt': '20021231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
코스코건설   2002
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '코스코건설', 'searchRpcdIsBgDt': '20020101', 'searchRpcdIsEdDt': '20021231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
아주건설(주)외2개사   2002
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '아주건설(주)외2개사', 'searchRpcdIsBgDt': '20020101', 'searchRpcdIsEdDt': '20021231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
예서건설㈜   2002
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '예서건설

...

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)Expect

Exception in thread Exception in thread Thread-95:
Traceback (most recent call last):
  File "/Users/waynehwang/opt/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-52Exception in thread Thread-81:
Traceback (most recent call last):
  File "/Users/waynehwang/opt/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-61:
Traceback (most recent call last):
  File "/Users/waynehwang/opt/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-79:
Traceback (most recent call last):
  File "/Users/waynehwang/opt/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-53:
Traceback (most recent call last):
  File "/Users/waynehwang/opt/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Thread-77:
Traceback (most recent call last):
  File "/Users/waynehwang/opt/anaconda3/lib/python3.8/threading.py", line 932, i


self.run()
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-12-2167087db3d6>", line 15, in threadReq
  File "<ipython-input-12-2167087db3d6>", line 15, in threadReq
self._target(*self._args, **self._kwargs)self.run()AttributeError
  File "<ipython-input-12-2167087db3d6>", line 15, in threadReq
self._target(*self._args, **self._kwargs)  File "/Users/waynehwang/opt/anaconda3/lib/python3.8/threading.py", line 870, in run
:         
  File "<ipython-input-12-2167087db3d6>", line 15, in threadReq
self._target(*self._args, **self._kwargs)AttributeError: 'NoneType' object has no attribute 'empty'
  File "/Users/waynehwang/opt/anaconda3/lib/python3.8/threading.py", line 870, in run

    
AttributeError  File "<ipython-input-12-2167087db3d6>", line 15, in threadReq
  File "/Users/waynehwang/opt/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)self._target(*self._args, **self._kwargs)
  File "<ipython-input-12-2167087db3d6>

'NoneType' object has no attribute 'empty'
: 'NoneType' object has no attribute 'empty'

AttributeErrorAttributeError: 'NoneType' object has no attribute 'empty'
AttributeError: 'NoneType' object has no attribute 'empty'
: 'NoneType' object has no attribute 'empty'
'NoneType' object has no attribute 'empty'
: 'NoneType' object has no attribute 'empty'
: 'NoneType' object has no attribute 'empty'AttributeErrorAttributeError: 'NoneType' object has no attribute 'empty'
: 'NoneType' object has no attribute 'empty'
AttributeErrorAttributeError: 'NoneType' object has no attribute 'empty'
AttributeError: 'NoneType' object has no attribute 'empty'
'NoneType' object has no attribute 'empty'
: 'NoneType' object has no attribute 'empty'
AttributeError: 'NoneType' object has no attribute 'empty'
  File "<ipython-input-12-2167087db3d6>", line 15, in threadReq
AttributeError: 'NoneType' object has no attribute 'empty'
AttributeErrorAttributeError: 'NoneType' object has no attribute 'empty'

    self

Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)


Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)


Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)

Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)






Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)



Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)


Expecting value: line 1 column 1 (char 0)

Expecting value: line 1 col

In [15]:
resultDfLastMerged

""


>totalCount 11
0 no pageNo
x :  5  g_totalCount :  -989  numOfRows :  1000
명화건설(주)   2016
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '명화건설(주)', 'searchRpcdIsBgDt': '20160101', 'searchRpcdIsEdDt': '20161231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
디엘종합건설(주)   2009
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '디엘종합건설(주)', 'searchRpcdIsBgDt': '20090101', 'searchRpcdIsEdDt': '20091231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
(주)대우건설, 대창건설(주),(주)삼전건설   2011
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)대우건설, 대창건설(주),(주)삼전건설', 'searchRpcdIsBgDt': '20110101', 'searchRpcdIsEdDt': '20111231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
이화공영(주).우람종합건설(주)

In [16]:
cf.savedata(resultDfLastMerged, SITENAME,DATANAME,SERVICENAME)

품질검사성적서 등록 목록 save compled
>totalCount 3
0 no pageNo
x :  5  g_totalCount :  -997  numOfRows :  1000
명화건설(주)   2019
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '명화건설(주)', 'searchRpcdIsBgDt': '20190101', 'searchRpcdIsEdDt': '20191231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
디엘종합건설(주)   2012
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '디엘종합건설(주)', 'searchRpcdIsBgDt': '20120101', 'searchRpcdIsEdDt': '20121231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
(주)경화엔지니어링,(주)장산엔지니어링   2021
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)경화엔지니어링,(주)장산엔지니어링', 'searchRpcdIsBgDt': '20210101', 'searchRpcdIsEdDt': '20211231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Da

In [17]:
resultDfLastMerged.shape

>totalCount 6
0 no pageNo
x :  5  g_totalCount :  -994  numOfRows :  1000
명화건설(주)   2020
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '명화건설(주)', 'searchRpcdIsBgDt': '20200101', 'searchRpcdIsEdDt': '20201231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start


(0, 0)

>totalCount 1
0 no pageNo
x :  7  g_totalCount :  -999  numOfRows :  1000
대림종합건설(주), 대휘건설(주)   2017
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '대림종합건설(주), 대휘건설(주)', 'searchRpcdIsBgDt': '20170101', 'searchRpcdIsEdDt': '20171231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 1
0 no pageNo
x :  4  g_totalCount :  -999  numOfRows :  1000
(주)대우건설, 대창건설(주),(주)삼전건설   2014
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)대우건설, 대창건설(주),(주)삼전건설', 'searchRpcdIsBgDt': '20140101', 'searchRpcdIsEdDt': '20141231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 4
0 no pageNo
x :  1  g_totalCount :  -996  numOfRows :  1000
디엘종합건설(주)   2013
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '디엘종합건설(주)', 'searchRpcdIsBgDt': '20130101', 'searchRpcdIsEdDt': '20131231', 'sortField': '

1 page is empty
No Data
흥창건설   2002
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '흥창건설', 'searchRpcdIsBgDt': '20020101', 'searchRpcdIsEdDt': '20021231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
이화공영(주).우람종합건설(주)   2018
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '이화공영(주).우람종합건설(주)', 'searchRpcdIsBgDt': '20180101', 'searchRpcdIsEdDt': '20181231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 6
0 no pageNo
x :  1  g_totalCount :  -994  numOfRows :  1000
디엘종합건설(주)   2017
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '디엘종합건설(주)', 'searchRpcdIsBgDt': '20170101', 'searchRpcdIsEdDt': '20171231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
대림종합건설(주), 대휘건설(주)   2021
BASEPARAM  {

1 page is empty
No Data
(주효창   2012
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주효창', 'searchRpcdIsBgDt': '20120101', 'searchRpcdIsEdDt': '20121231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 2
0 no pageNo
x :  1  g_totalCount :  -998  numOfRows :  1000
디엘종합건설(주)   2021
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '디엘종합건설(주)', 'searchRpcdIsBgDt': '20210101', 'searchRpcdIsEdDt': '20211231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page is empty1 page scraping start

No Data
이화공영(주).우람종합건설(주)   2022
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '이화공영(주).우람종합건설(주)', 'searchRpcdIsBgDt': '20220101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
흥창건설   2007
BASEPARAM  {'serviceKey': 

1 page is empty
No Data
직접기초   2003
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '직접기초', 'searchRpcdIsBgDt': '20030101', 'searchRpcdIsEdDt': '20031231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 2
0 no pageNo
x :  5  g_totalCount :  -998  numOfRows :  1000
흥창건설   2011
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '흥창건설', 'searchRpcdIsBgDt': '20110101', 'searchRpcdIsEdDt': '20111231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
대한기업(주)   2004
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '대한기업(주)', 'searchRpcdIsBgDt': '20040101', 'searchRpcdIsEdDt': '20041231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
영원건설   2007
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB